# Check GPU devices

In [1]:
import torch
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


# python `import`s

In [2]:
import numpy as np
from pprint import pprint
from liberate import fhe

#######################
>>>> Using GPU backend
#######################


# Generate params

In [3]:
params = fhe.presets.params["gold"]
params["devices"] = [0]
pprint(params)

{'devices': [0],
 'logN': 16,
 'num_scales': None,
 'num_special_primes': 4,
 'scale_bits': 40}


# Generate CkksEngine

In [4]:
engine = fhe.CkksEngine(**params, verbose=True)

[2024-09-29 16:41:37.777031] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 16
        N				= 65,536
        Number of special primes	= 4
        Number of scales		= 34
        Cache folder			= '/home/test00/.pyenv/versions/LG_gpu/lib/python3.10/site-packages/liberate/gpu/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '1,660' bits out of available maximum '1,661' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516870657, 1099506515969, 1099521458177, 1099503894529, 1099522375681, 1099490000897, 1099523555329, 1099489607681, 1099525128193, 1099486855169, 1099526176769, 1099484889089, 1099529060353, 1099480956929, 1099535220737, 1099469684737, 1099536138241, 1099468767233, 10995

# Generate GPU Keys

In [5]:
sk = engine.create_secret_key()
pk = engine.create_public_key(sk)
evk = engine.create_evk(sk)
galk = engine.create_galois_key(sk)

# Save GPU Keys

In [6]:
engine.save(sk, "sk_gpu.pkl")
engine.save(pk, "pk_gpu.pkl")
engine.save(evk, "evk_gpu.pkl")
engine.save(galk, "galk_gpu.pkl")

# Load CPU Keys

In [7]:
sk_cpu = engine.load("sk_cpu.pkl")
pk_cpu = engine.load("pk_cpu.pkl")
evk_cpu = engine.load("evk_cpu.pkl")
galk_cpu = engine.load("galk_cpu.pkl")

# Test Encorypt with CPU Keys

In [8]:
m = np.random.uniform(-1, 1, size=engine.num_slots)
ct = engine.encorypt(m, pk_cpu)
m_ = engine.decrode(ct, sk_cpu)

print((m_/m).mean().real)

1.0000000000605207


# Test Mult with CPU Keys

In [9]:
m = np.random.uniform(-1, 1, size=engine.num_slots)
ct = engine.encorypt(m, pk_cpu)

ct_mult = engine.mult(ct, ct, evk_cpu)

m_mult_ = engine.decrode(ct_mult, sk_cpu)

print((m_mult_/(m*m)).mean().real)

1.0000005915746923


# Test rotate with CPU Keys

In [10]:
m = np.random.uniform(-1, 1, size=engine.num_slots)
ct = engine.encorypt(m, pk_cpu)

idx = np.random.randint(-2**15, 2**15)
ct_galk = engine.rotate_galois(ct, gk=galk_cpu, delta=idx)

m_galk_ = engine.decrode(ct_galk, sk_cpu)

print((m_galk_/np.roll(m, idx)).mean().real)

1.0000000000592717
